"""
Machine Learning Homework 4
Done by:
Mariana Santana 106992
Pedro Leal 106154
LEIC-A
"""

#### Consider the breast_cancer dataset data = datasets.load_breast_cancer()  with binary target variable y=‘malignant’. Split it 70% for training and 30% for testing.

In [21]:
"""
General imports and variables for all exercises; run this cell before any other
"""
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import Ridge

data = load_breast_cancer()
X, y = data.data, data.target

#### 1) Perform logistic regression and indicate the accuracy. 

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

log_reg = LogisticRegression(max_iter=10000, random_state=42)
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")

Accuracy: 0.9766081871345029


#### 2) Perform EM clustering on the training data set with different number k of clusters. Evaluate the quality of the clusterings using Silhouette. Is the number of clusters correlated with the quality of clustering? Which is the optimal k? 

In [23]:
k_values = range(2, 10)

silhouette_scores = []

for k in k_values:
    gmm = GaussianMixture(n_components=k, random_state=42)
    gmm.fit(X_train)
    labels = gmm.predict(X_train)
    score = silhouette_score(X_train, labels)
    silhouette_scores.append(score)

optimal_k = k_values[np.argmax(silhouette_scores)]
print(optimal_k)

2


#### 3) Map the test set into probability values of the k-clusters. If you have a data point represented by a vector of dimension d, you will map it into a vector of dimension: prob=em_model.predict_proba(X)

In [24]:
em_model = GaussianMixture(n_components=optimal_k, random_state=42)
em_model.fit(X_train)

probabilities = em_model.predict_proba(X_test)
print(probabilities.shape, probabilities)

(171, 2) [[8.45317965e-013 1.00000000e+000]
 [1.00000000e+000 7.98725997e-101]
 [9.99999903e-001 9.74364029e-008]
 [4.79691619e-023 1.00000000e+000]
 [3.29595693e-028 1.00000000e+000]
 [1.00000000e+000 5.15118097e-058]
 [1.00000000e+000 1.05400217e-207]
 [1.00000000e+000 3.58578868e-013]
 [1.00000000e+000 1.08956508e-014]
 [5.37102624e-011 1.00000000e+000]
 [2.38594015e-010 1.00000000e+000]
 [1.00000000e+000 5.87241433e-017]
 [4.51319532e-006 9.99995487e-001]
 [5.93434266e-001 4.06565734e-001]
 [3.97311911e-011 1.00000000e+000]
 [1.00000000e+000 1.16758100e-041]
 [6.60425533e-007 9.99999340e-001]
 [1.49677648e-058 1.00000000e+000]
 [1.74792741e-112 1.00000000e+000]
 [1.00000000e+000 6.27211373e-096]
 [8.38060972e-005 9.99916194e-001]
 [1.28295310e-016 1.00000000e+000]
 [1.00000000e+000 3.42801970e-171]
 [1.64521942e-036 1.00000000e+000]
 [6.85127259e-029 1.00000000e+000]
 [1.84222404e-027 1.00000000e+000]
 [9.68653958e-010 9.99999999e-001]
 [4.04137455e-039 1.00000000e+000]
 [1.7538027

#### 4) Perform logistic regression on the mapped data set with the labels of the original test set. Indicate now the accuracy. Is there a relation between the number of clusters, the cluster evaluation and the accuracy of the logistic regression model?

In [25]:
X_mapped = probabilities

log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_mapped, y_test)

y_pred = log_reg.predict(X_mapped)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9239766081871345


#### 5) Train an RBF network using the clustering with optimal k  from 2).

In [26]:
centers = em_model.means_

X_rbf_transformed = rbf_kernel(X_test, centers)

rbf_model = Ridge(alpha=1.0)
rbf_model.fit(X_rbf_transformed, y_test)

y_rbf_pred = rbf_model.predict(X_rbf_transformed)

y_rbf_pred_binary = (y_rbf_pred >= 0.5).astype(int)

rbf_accuracy = accuracy_score(y_test, y_rbf_pred_binary)
print(rbf_accuracy)

0.631578947368421
